In [69]:
import bs4 as bs
import requests
import os
import re
import sys
import json

In [70]:
def writeLinksJson(article, append=False):
    file = 'dataBase/articles.json'
    if append:
        with open(file, 'a+') as jsonOut:
            json.dump(article, jsonOut, indent=2, ensure_ascii=False)
    else:
        with open(file, 'w+') as jsonOut:
            json.dump(article, jsonOut, indent=2, ensure_ascii=False)
            # ensure_ascii=False : pour bien afficher les caractéres accentueux

In [71]:
def readLinksJson():
    file = 'dataBase/articles.json'
    if os.path.isfile(file):
        with open(file) as jsonIn:
            data = json.load(jsonIn)
            return data
    else:
        print("I can't find ", file)

In [72]:
def common(a, b):
    c = [value for value in a if value in b]
    return c

def common2(a, b):
    return set(a).intersection(b)

In [73]:
def articleLinkIdGenerator(online=False):
    # on a utilisé le online=false car si on est pas en ligne on pourra
    # utiliser la dataBase pour afficher les resultats voulue
    url = 'https://pagellapolitica.it/'
    uriPage = 'dichiarazioni/verificato?page='
    jsonFile = 'dataBase/articles.json'
    articles = {}
    if online:
        for i in range(sys.maxsize):
            pageNumber = i
            pageNumber = str(pageNumber)
            pageAddress = url + uriPage + pageNumber

            resp = requests.get(pageAddress)
            if resp:
                source = requests.get(pageAddress).text
                soup = bs.BeautifulSoup(source, "lxml")
                tables = soup.findAll('div', {'class': 'col-lg-3 mb-7'})
                for table in tables:
                    uriArticleID = table.find(
                        'a', {'class': 'statement-link'})['href']
                    idPattern = re.search(r'/([0-9]+)/', uriArticleID)
                    idNumber = idPattern.group(1)
                    articles[idNumber] = {
                        'url': url+uriArticleID,
                    }
                if idNumber == str(215):
                    break
            if idNumber == str(215):
                break

        with open(jsonFile, 'w') as file:
            # la sortie de Json est une variable ici c'est a
            json.dump(articles, file, indent=2, ensure_ascii=False)
    else:
        with open(jsonFile) as file:
            articles = json.load(file)
    return articles

In [74]:
def extractInfoArticle(id, online=False):

    article = {}
    url = 'https://pagellapolitica.it/'
    if online:
        uriArticle = 'dichiarazioni/'
        address = url + uriArticle + id
        print(address)
        resp = requests.get(address)
        if resp:
            source = requests.get(address).text
            soup = bs.BeautifulSoup(source, "lxml")

            # auteur
            p = soup.find(
                'p', {'class': 'h4 mb-1 px-2 text-dark font-weight-light'})
            author = p.find('a', {'class': 'u-link-muted'}).text.strip()

            # le titre au complet quand on rentre dans l'article
            divFullBody = soup.find('div', {'class': 'col-lg-9 mb-9 mb-lg-0'})
            if divFullBody:
                # veracity
                # h2 mark-ni
                # h2 mark-vero
                # h2 mark-ceri
                # h2 mark-pinocchio
                # h2 mark-panzana

                veracities = ["mark-ni", "mark-vero",
                              "mark-ceri", "mark-pinocchio", "mark-panzana"]
                divTitle = divFullBody.find('div', {'class': 'mb-2 mt-2 px-2'})
                divTitlePH2Px2 = divFullBody.find(
                    'p', {'class': 'h2 px-2'})
                if divTitle:
                    fullTitle = divTitle.find(
                        'span', {'class': 'h2'}).text.strip()

                    spanVeracity = divTitle.find(
                        'span', {'class': 'h2'})
                    thisVeracity = spanVeracity['class']
                    veracity = common(thisVeracity, veracities)[0]
                elif divTitlePH2Px2:
                    fullTitle = divTitlePH2Px2.find(
                        'span', {'class': 'text-darker'}).text.strip()
                    spanVeracity = divTitlePH2Px2.find(
                        'span', {'class': 'text-darker'})
                    thisVeracity = spanVeracity['class']
                    veracity = common(thisVeracity, veracities)[0]

                else:
                    # dans les anciens artictles le fullText n'existe pas donc
                    # si il ne le trouve pas il affiche le msg
                    fullTitle = "No fullTitle"
                    veracity = "No veracity"

            # les claims
            pclaimPX3 = soup.find(
                'p', {'class': 'lead px-3 py-3 bg-light g-brd-around g-brd-lightblue'})
            pclaimPX2 = soup.find('p', {'class': 'h2 px-2'})
            if pclaimPX3:
                claim = pclaimPX3.text.strip()
            elif pclaimPX2:
                claim = pclaimPX2.find(
                    'span', {'class': 'text-darker'}).text.strip()

            # les dates(publication,origine) elles sont dans la méme classe d'ou l'utilisation
            # de divDates[0] pour la publication et divDates[1] pour l'origine
            divDate = soup.find('div', {'class': 'card-body pt-0 px-2'})
            divDates = divDate.findAll('div', {'class': 'col-lg-2 text-left'})
            datePublished = divDates[0].find(
                'span', {'class': 'text-dark'}).text.strip()
            dateOrigin = divDates[1].find(
                'span', {'class': 'text-dark'}).text.strip()

            # statement source
            divReferredLinks = divDate.find('div', {'class': 'col-lg-4'})
            statementSource = divReferredLinks.find(
                'a', {'class': 'u-link-muted'})['href']

            # le join est pour concaténé les textes des balises P pour les mettre en un seul txt
            divMainArticle = soup.find('div', {'id': 'analisi-content'})
            listOfPInMainArticle = divMainArticle.findAll('p')
            mainText = [text.text.strip() for text in listOfPInMainArticle]
            mainArticle = ' '.join(mainText)

            # les liens de reference dans le main text
            # listOfAllLinks = divMainArticle.findAll('a')
            for link in divMainArticle.findAll('a', href=True):
                print(link)
                if link['href']:
                    print("link:",link['href'])
            #listOfAllLinksHref = [link['href'] for link in divMainArticle.findAll('a') if link['href']]

            # Tags
            if divFullBody.find('div', {'class': 'px-2 u-space-2-top'}):
                divTags = divFullBody.find(
                    'div', {'class': 'px-2 u-space-2-top'})
                tags = divTags.findAll(
                    'a', {'class': 'btn btn-sm btn-light u-btn-light transition-3d-hover rounded-0 mb-2'})

                tagList = []
                for tag in tags:
                    tagList.append(tag.text.strip())

    else:
        data = readLinksJson()
        # pour l'ajout d'un element dans un dictionnaire exsitant ,**dic[2] recupére
        # tout les anciens elements du dic
        data[id] = {
            **data[id],
            # le site de fact checking
            "source": "source not found",
            "claim": "claim not found",
            "body": "mainArticle not found",                     # le text de l'article
                    "referred_links": "listOfAllLinksHref not found",    # tous les liens dans le texte
                    "title": "fullTitle not found",                      # le titre de l'article
                    "date": "dateOrigin not found",                      # date de la claim
                    "url": "address not found",  # url de l'article
                    "tags": "tags not found",                          # les mots cles
                    "author": "author not found",                       # auteur de la claim
                    "datePublished": "datePublished not found",
                    "rating_value": "rating_value not found",          # la valeur de la veracite
                    "statementSource": "statementSource not found",
                    # les entities nomes qui est extraite de la claim
                    "claim_entities": "claim_entities not found",
                    # les entities nomes qui est extraite de l'article
                    "body_entities": "body_entities not found",
                    # parmi les tages, les entities nomes a partir de la tag
                    "keyword_entities": "keyword_entities not found",
                    # les entities nomes a partir de l'auteur de la claim
                    "author_entities": "author_entities not found",
                    "review_author": "review_author not found"         # l'auteur de l'article

        }

    article = {
        id: {
            "source": url,                           # le site de fact checking
            "claim": claim,
            "body": mainArticle,                     # le text de l'article
            "referred_links": listOfAllLinksHref,    # tous les liens dans le texte
            "title": fullTitle,                      # le titre de l'article
            "date": dateOrigin,                      # date de la claim
            "url": address,  # url de l'article
            "tags": tagList,                          # les mots cles
            "author": author,                       # auteur de la claim
            "datePublished": datePublished,
            "rating_value": veracity,          # la valeur de la veracite
            "statementSource": statementSource,
            # les entities nomes qui est extraite de la claim
            "claim_entities": "claim_entities not found",
            # les entities nomes qui est extraite de l'article
            "body_entities": "body_entities not found",
            # parmi les tages, les entities nomes a partir de la tag
            "keyword_entities": "keyword_entities not found",
            # les entities nomes a partir de l'auteur de la claim
            "author_entities": "author_entities not found",
            "review_author": "review_author not found"         # l'auteur de l'article

        }
    }

    return article

In [75]:
extractInfoArticle("1327", online=True)

https://pagellapolitica.it/dichiarazioni/1327
<a href="http://www.quirinale.it/qrnw/statico/costituzione/costituzione.htm#Principi">l’articolo 117 comma 4 della Costituzione</a>
link: http://www.quirinale.it/qrnw/statico/costituzione/costituzione.htm#Principi
<a href="http://www.salute.gov.it/ministero/sezMinistero.jsp?label=ssn">la competenza concorrente  in ambito sanitario</a>
link: http://www.salute.gov.it/ministero/sezMinistero.jsp?label=ssn
<a href="http://www.regione.piemonte.it/pianosanitario/dwd/piano_socio_sanitario.pdf">La riforma sanitaria, per la Regione Piemonte, prevede tagli ulteriori alla spesa sanitaria, diminuendo il numero di ospedali e servizi al cittadino</a>
link: http://www.regione.piemonte.it/pianosanitario/dwd/piano_socio_sanitario.pdf
<a href="http://www.salute.gov.it/portale/news/p3_2_1_1_1.jsp?lingua=italiano&amp;menu=notizie&amp;p=dalministero&amp;id=936">Come visibile sul sito del Ministero della Salute, il Piemonte è l’unica Regione settentrionale impegn

NameError: name 'listOfAllLinksHref' is not defined

In [41]:
def pageLinks(pageNumber):
    url = 'https://pagellapolitica.it/'
    uri = 'dichiarazioni/verificato?page='
    pageAddress = url + uri + pageNumber
    resp = requests.get(pageAddress)
    if resp:
        source = requests.get(pageAddress).text
        soup = bs.BeautifulSoup(source, "lxml")
        tables = soup.findAll('div', {'class': 'col-lg-3 mb-7'})
        pageLinks = []
        for table in tables:
            link = table.find('a', {'class': 'statement-link'})['href']
            articleLink = url+link
            pageLinks.append(articleLink)
    return pageLinks

In [29]:
pageLinks = pageLinks("1")
for i in pageLinks:
    print(i)

https://pagellapolitica.it/dichiarazioni/8548/no-nel-2008-la-germania-non-ha-dato-300-miliardi-alle-sue-banche
https://pagellapolitica.it/dichiarazioni/8547/emiliano-impreciso-sui-pugliesi-militari-e-su-quelli-fuggiti-dal-nord
https://pagellapolitica.it/dichiarazioni/8546/renzi-esagera-sui-detenuti-nelle-carceri-italiane-in-un-verso-e-nellaltro
https://pagellapolitica.it/dichiarazioni/8545/salvini-esagera-un-po-sul-pil-di-lombardia-e-veneto
https://pagellapolitica.it/dichiarazioni/8544/che-cosa-dicono-i-primi-dati-sui-morti-con-il-coronavirus-in-italia
https://pagellapolitica.it/dichiarazioni/8543/la-germania-e-meno-vecchia-anche-grazie-ai-migranti
https://pagellapolitica.it/dichiarazioni/8542/la-sanita-italiana-e-davvero-uneccellenza-mondiale
https://pagellapolitica.it/dichiarazioni/8541/di-maio-e-la-geografia-della-quarantena
https://pagellapolitica.it/dichiarazioni/8540/e-vero-le-mascherine-sono-inutili-per-chi-e-sano
https://pagellapolitica.it/dichiarazioni/8539/laumento-delliva-in

In the welcome page we see only 9 articles but when we see the HTML code, there are 16 article's link.

In [30]:
print(len(pageLinks))

16


In [42]:
def idNumber(link):
    idPattern = re.search(r'/([0-9]+)/', link)
    idNumber = idPattern.group(1)
    return idNumber

In [43]:
#for pageNumber in range(42, 174):
pageURLs = pageLinks("1")
for page in pageURLs:
    id = idNumber(page)
    lastArticle = extractInfoArticle(id, online=True)
    #ter.writeLinksJson(lastArticle, append=True)
    print('rating_value = ', lastArticle[id]["rating_value"])
    print("\n")

https://pagellapolitica.it/dichiarazioni/8548
rating_value =  mark-ni


https://pagellapolitica.it/dichiarazioni/8547
rating_value =  mark-ni


https://pagellapolitica.it/dichiarazioni/8546
rating_value =  mark-ni


https://pagellapolitica.it/dichiarazioni/8545
rating_value =  mark-ceri


https://pagellapolitica.it/dichiarazioni/8544
rating_value =  mark-vero


https://pagellapolitica.it/dichiarazioni/8543
rating_value =  mark-ceri


https://pagellapolitica.it/dichiarazioni/8542
rating_value =  mark-ceri


https://pagellapolitica.it/dichiarazioni/8541
rating_value =  mark-ceri


https://pagellapolitica.it/dichiarazioni/8540
rating_value =  mark-vero


https://pagellapolitica.it/dichiarazioni/8539
rating_value =  mark-ceri


https://pagellapolitica.it/dichiarazioni/8538
rating_value =  mark-ni


https://pagellapolitica.it/dichiarazioni/8537
rating_value =  mark-ni


https://pagellapolitica.it/dichiarazioni/8536
rating_value =  mark-ni


https://pagellapolitica.it/dichiarazioni/8535
rati

In [44]:
def findPageNumberOfID(id):
    for pageNumber in range(1, 200):
        print("Page:", pageNumber)
        pageURLs = pageLinks(str(pageNumber))
        for page in pageURLs:
            idWanted = idNumber(page)
            if idWanted == id:
                print("ID", id, "is in the page:", pageNumber)
                break
        if idWanted == id:
            break

In [45]:
findPageNumberOfID("1269")

Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
Page: 16
Page: 17
Page: 18
Page: 19
Page: 20
Page: 21
Page: 22
Page: 23
Page: 24
Page: 25
Page: 26
Page: 27
Page: 28
Page: 29
Page: 30
Page: 31
Page: 32
Page: 33
Page: 34
Page: 35
Page: 36
Page: 37
Page: 38
Page: 39
Page: 40
Page: 41
Page: 42
Page: 43
Page: 44
Page: 45
Page: 46
Page: 47
Page: 48
Page: 49
Page: 50
Page: 51
Page: 52
Page: 53
Page: 54
Page: 55
Page: 56
Page: 57
Page: 58
Page: 59
Page: 60
Page: 61
Page: 62
Page: 63
Page: 64
Page: 65
Page: 66
Page: 67
Page: 68
Page: 69
Page: 70
Page: 71
Page: 72
Page: 73
Page: 74
Page: 75
Page: 76
Page: 77
Page: 78
Page: 79
Page: 80
Page: 81
Page: 82
Page: 83
Page: 84
Page: 85
Page: 86
Page: 87
Page: 88
Page: 89
Page: 90
Page: 91
Page: 92
Page: 93
Page: 94
Page: 95
Page: 96
Page: 97
Page: 98
Page: 99
Page: 100
Page: 101
Page: 102
Page: 103
Page: 104
Page: 105
Page: 106
Page: 107
Page: 108
Page: 109
Page: 110
Page: 11

In [6]:
import pandas as pd
parameters = ['data-prix','data-codepostal','data-idagence','data-idannonce','data-nb_chambres','data-nb_pieces','data-surface','data-typebien']
df_f = pd.DataFrame()
for par in parameters:
    l = []
    for el in em_box:
        j = el[par]
        l.append(j)
    l = pd.DataFrame(l, columns = [par])
    df_f = pd.concat([df_f,l], axis = 1)

In [7]:
print(df_f)

Empty DataFrame
Columns: [data-prix, data-codepostal, data-idagence, data-idannonce, data-nb_chambres, data-nb_pieces, data-surface, data-typebien]
Index: []
